(compressed-air-label)=

# Compressed Air Production

```{figure} /figures/Compressor.svg
---
name: compressor-flowsheet
---
Abstract model representation of a Compressor.
```

## Introduction

In this example we are investigating the provision of compressed air. Compressed air is often required in technical
systems, e.g. for process control, or even as storage medium, i.e. in Compressed Air Energy Storage (CAES). To generate
compressed air a compressor is required, which will (in the best case) compress the air in an isentropic process. The
isentropic process is characterized by the fact, that the entropy does not change.

We can model such a process using CoolProp. For example, we want to compress 1.5 kg/s of air from the ambient state to
a pressure of 10 bar. To calculate the energy consumed in the isentropic process, the energy balance introduced in eq.
{eq}`pipe-flow-energy-balance` can be simplified by removing the heat transfer term:

```{math}
:label: compressor-energy-balance
\dot W = \dot m \cdot \left( h_2 - h_1\right)
```

In [ ]:
from CoolProp.CoolProp import PropsSI


T_amb = 20  # °C
p_amb = 1.01325  # bar
fluid = "air"
dot_m = 1.5

h_amb = PropsSI("H", "T", T_amb + 273.15, "P", p_amb * 1e5, fluid)
s_amb = PropsSI("S", "T", T_amb + 273.15, "P", p_amb * 1e5, fluid)

p_1 = p_amb
T_1 = T_amb

p_2 = 10

h_1 = h_amb
s_1 = s_amb
s_2 = s_1

h_2 = PropsSI("H", "S", s_1, "P", p_2 * 1e5, fluid)

dot_W = dot_m * (h_2 - h_1)

dot_W

We can define the exergy balance for the compressor at temperatures above the ambient temperature in the eqs.
{eq}`compressor-fuel-exergy` and {eq}`compressor-product-exergy`. With that definition it is possible to determine fuel,
product and exergy destruction.

```{math}
:label: compressor-fuel-exergy
\dot E_\text{F} = \dot W
```

```{math}
:label: compressor-product-exergy
\dot E_\text{P} = \dot{E}_\text{out}^\text{PH} - \dot{E}_\text{in}^\text{PH}
```

In [ ]:
from utilities import calc_physical_exergy


exergy_fuel = dot_W

ex_1 = calc_physical_exergy(p_1 * 1e5, h_1, p_amb * 1e5, T_amb + 273.15, fluid)
ex_2 = calc_physical_exergy(p_2 * 1e5, h_2, p_amb * 1e5, T_amb + 273.15, fluid)

exergy_product = dot_m * (ex_2 - ex_1)

exergy_destruction = exergy_fuel - exergy_product

exergy_destruction

First of all, we see that the exergy destruction is zero, which verifies our calculation: Since the isentropic process
is adiabatic and reversible we have an thermodynamically speaking ideal process. Since such a component can only exist
in theory, a reference figure is used to define how close an actual compressor is to the theoretical optimum: the
isentropic efficiency {math}`\eta_\text{s,cmp}` in eq. {eq}`compressor-isentropic-efficiency`:

```{math}
:label: compressor-isentropic-efficiency
\eta_\text{s,cmp} = \frac{h \left( p_2, s_1\right) - h_1}{h_2 - h_1}
```

## Exercises 1

1. Assume you want to compress the same amount of air to the same pressure as in the example before:
   - What power does a compressor with an isentropic efficiency of 85 % consume?
   - How large is the exergy destruction?
   - How and why does the product exergy change?    

## Solution 1

To calculate the outlet state of the compressor with the isentropic efficiency of 85 % we have to first reorder the eq.
{eq}`compressor-isentropic-efficiency` in a way, that the unknown outlet enthalpy can be calculated:

```{math}
:label: compressor-isentropic-efficiency-reorder
h_2 = \frac{h \left( p_2, s_1\right) - h_1}{\eta_\text{s,cmp}} + h_1
```

Then we create a function to calculate the enthalpy after isentropic compression {math}`h \left( p_2, s_1\right)` and
call it with the respective parameters.

In [ ]:
def isentropic_outlet_enthalpy(p_1, h_1, p_2, fluid):
    s_1 = PropsSI("S", "P", p_1 , "H", h_1, fluid)
    h_2s = PropsSI("H", "P", p_2, "S", s_1, fluid)
    return h_2s


eta_s = 0.85

h_2_real = (isentropic_outlet_enthalpy(p_1 * 1e5, h_1, p_2 * 1e5, fluid) - h_1) / eta_s + h_1

dot_W_real = dot_m * (h_2_real - h_1)

dot_W_real / dot_W - 1

In [ ]:
from myst_nb import glue
glue("compressor-work-increase", round((dot_W_real / dot_W - 1) * 100, 2), display=False)

We can see how the work required increases by about {glue:text}`compressor-work-increase` %. Now let's have a look at
the exergy analysis. We can calculate the balance with the updated outlet state and have a look at the exergy
destruction:

In [ ]:
exergy_fuel_real = dot_W_real

ex_2_real = calc_physical_exergy(p_2 * 1e5, h_2_real, p_amb * 1e5, T_amb + 273.15, fluid)

exergy_product_real = dot_m * (ex_2_real - ex_1)

exergy_destruction_real = exergy_fuel_real - exergy_product_real

exergy_destruction_real

In [ ]:
glue("compressor-exergy-destruction", round(exergy_destruction_real), display=False)
glue("compressor-work-change", round(dot_W_real - dot_W), display=False)

The exergy destruction is at {glue:text}`compressor-exergy-destruction` W, the change of work transferred by the
compressor (compared to the ideal compressor) is {glue:text}`compressor-work-change` W. This may seem odd at the first
glance. If the power input to the process has increased stronger than the exergy destruction, more product exergy must
be available than before. We can verify this:

In [ ]:
exergy_product_real - exergy_product

## Overall process product

We have learned how to run energy and exergy balances on a single compressor and found, that a less efficiency 
compressor consumes more power for the provision of the same mass flow at a specific pressure (obviously), but also that
the product exergy increases the less efficient the compressor is.

This seems very counter intuitive as we always provide 1.5 kg/s compressed air at a pressure of 10 bar. However, we have
not yet looked at the outlet temperature of the compressor. We can do that by calculating the temperature at the outlet
pressure and the respective enthalpy.

In [ ]:
T_2_is = PropsSI("T", "P", p_2 * 1e5, "H", h_2, fluid)
T_2_is - 273.15

In [ ]:
T_2 = PropsSI("T", "P", p_2 * 1e5, "H", h_2_real, fluid)
T_2 - 273.15

It becomes clear, that the outlet temperature rises with a lower isentropic efficiecy of the compressor. Obviously, our
total exergy changes, but we want to have a look at how the shares of phyiscal and thermal exergy change.

### Intermediate Exercise 2

2. Compare the mechanical and thermal exergy for the compressed air provided by the ideal and the real compressor. What
   is strinking?

### Intermediate Solution 2

We can make use of the exergy splitting to calculate thermal and mechanical exergy and specifically look at the 
mechanical exergy first: It does NOT change.

In [ ]:
from utilities import calc_splitted_physical_exergy


ex_2_ideal_therm, ex_2_ideal_mech = calc_splitted_physical_exergy(p_2 * 1e5, h_2, p_amb * 1e5, T_amb + 273.15, fluid)
ex_2_therm, ex_2_mech = calc_splitted_physical_exergy(p_2 * 1e5, h_2_real, p_amb * 1e5, T_amb + 273.15, fluid)


ex_2_mech - ex_2_ideal_mech

That means the change in product exergy is purely due to a change in thermal exergy:

In [ ]:
dot_m * (ex_2_therm - ex_2_ideal_therm)

In [ ]:
exergy_product_real - exergy_product

What does this mean about the overall process product exergy? In our technical system we want to provide pneumatic 
energy. We cannot make use of the high temperature for this specific reason. Of course, one could use that thermal
exergy to provide heat to other processes, but for now we want to focus on the provison of pressurized air. Thinking
at this, the value of our process lies in the pressure of the air, not in the temperature. Therefore we want to 
investigate more on this by defining a virtual process, which cools the pressurized air back to the ambient temperature
without changing the pressure as illustrated in {numref}`compressor-virtual-cooling-flowsheet`.

```{figure} /figures/Compressor_virtual_cooling.svg
---
name: compressor-virtual-cooling-flowsheet
---
Process representation of the compressor with virtual cooling of the compressed air.
```

## Exercise 3

3. Balance the virtual process according to the flowsheet above.
   - Between the states 1 and 3
     - How does the enthalpy change?
     - How do mechanical and thermal exergy change?
   - How does the heat transferred in the virtual process from 2 to 3 compare to the change in thermal exergy?

## Solution 3

The temperature rises together with the pressure. However, we cannot really make use of the temperature in pneumatic
systems or in compressed air energy storage. 



```{math}
    :label: eq:compressed-air-information
    T_3 = T_1\\
    p_3 > p_1
```

Applying energy analysis to the overall system neglecting the influence of
potential and kinetic energy to the thermal system, we can calculate the total
amount of energy transferred over the system boundaries $\dot E$:

```{math}
    :label: eq:compressed-air-energy
    \dot E =\dot m \cdot \left( h_3 - h_1\right)
```

However, we can see that the change in enthalpy is nearly zero, since $h_1$ and
$h_3$ are almost equal.

```{math}
    :label: eq:compressed-air-enthalpy-equality
    h_\left(p_1, T_1\right) \sim h_\left(p_3, T_3\right)
```

We can easily verify this claim:


In [ ]:
from CoolProp.CoolProp import PropsSI

T_amb = 30  # °C
p_amb = 1.01325  # bar
fluid = "air"

p_3 = 5  # bar

p_amb_SI = p_amb * 1e5  # Pa
p_3_SI = p_3 * 1e5  # Pa
T_amb_SI = T_amb + 273.15  # K

h_1 = PropsSI("H", "P", p_amb_SI, "T", T_amb_SI, fluid)
h_3 = PropsSI("H", "P", p_3_SI, "T", T_amb_SI, fluid)

print(f"Change in enthalpy: {round(h_3 - h_1)} J/kg")

It even looks as if energy has been produced by our setup, since the change in enthalpy is
negative. We can see, how the enthalpy of air changes with pressure given 
isothermal process:

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

p_ref = 1e3  # Pa
p_max = 1e6  # Pa
p_range = np.linspace(p_ref, p_max)  # Pa
T_ref = 303.15  # K

h_ref = PropsSI("H", "P", p_ref, "T", T_ref, fluid)
h = PropsSI("H", "P", p_range, "T", T_ref, fluid)

fig, ax = plt.subplots(1)

ax.plot(p_range, h - h_ref)
ax.set_xlabel("$\Delta p$ in $Pa$")
ax.set_ylabel("$\Delta h$ in $J/kg$")

ax.set_axisbelow(True)
ax.grid(color='gray', linestyle='dashed')

plt.tight_layout()

For ideal gases, the change in enthalpy will be exactly zero, as the enthalpy
does not depend on change in pressure but only on change in temperature:

```{math}
    :label: eq:compressed-air-enthalpy-ideal-gas
    h\left(T\right) - h\left(T_\text{ref}\right) = c_\text{p}\left(T\right) \cdot \left(T - T_\text{ref}\right)
```

```{math}
    :label: eq:compressed-air-enthalpy-equality2
    h_\left(p_1, T_1\right) \sim h_\left(p_3, T_3\right)
```

To visualize the energy stored in the pressurized air, we can apply the exergy
method and split the exergy in its thermal and mechanical shares.

In [ ]:
from tespy.connections import Connection
from tespy.components import Sink, Source
from tespy.networks import Network


Network(["air"])


ref = Connection(Source("Dummysource1"), "out1", Sink("Dummysink1"), "in1")
c = Connection(Source("Dummysource2"), "out1", Sink("Dummysink2"), "in1")

ref.fluid.val = {"air": 1}
ref.p.val_SI = p_ref
ref.T.val_SI = T_ref
ref.h.val_SI = PropsSI("H", "P", p_ref, "T", T_ref, "air")
ref.s.val_SI = PropsSI("S", "P", p_ref, "T", T_ref, "air")

ref.get_physical_exergy(p_ref, T_ref)
print(f"Mechanical exergy at reference state: {round(ref.ex_mech)} J/kg")
# print(f"Thermal exergy at reference state: {round(ref.ex_therm)} J/kg")

c.fluid.val = {"air": 1}
c.T.val_SI = T_ref

ex = []

for p in p_range:
    c.p.val_SI = p
    c.h.val_SI = PropsSI("H", "P", p, "T", T_ref, "air")
    c.s.val_SI = PropsSI("S", "P", p, "T", T_ref, "air")

    c.get_physical_exergy(p_ref, T_ref)
    ex += [c.ex_mech]
    

ax.plot(p_range, ex)    
display(fig)
plt.close(fig)

With a closer look at the compressor assuming isentropic compression, we see that
the physical exergy $e^\text{PH}$ of the compressed air is identical to the 
isentropic compression energy $w = h_\text{is} - h$:

In [ ]:
s_ref = PropsSI("S", "P", p_ref, "T", T_ref, "air")
h_is = PropsSI("H", "P", p_range, "S", s_ref, "air")
T_is = PropsSI("T", "P", p_range, "S", s_ref, "air")

ex_therm = []
ex_mech = []
ex = []

for p, T in zip(p_range, T_is):
    c.h.val_SI = PropsSI("H", "P", p, "T", T, "air")
    c.s.val_SI = PropsSI("S", "P", p, "T", T, "air")
    c.p.val_SI = p
    c.get_physical_exergy(p_ref, T_ref)
    ex_therm += [c.ex_therm]
    ex_mech += [c.ex_mech]
    ex += [c.ex_physical]
    

fig, ax = plt.subplots(1)

ax.set_xlabel("$\Delta p$ in $Pa$")
ax.set_ylabel("$\Delta h$ in $J/kg$")

ax.set_axisbelow(True)
ax.grid(color='gray', linestyle='dashed')

plt.tight_layout()
    
ax.plot(p_range, h_is - ref.h.val_SI, "x-", label="$w_{is}$")
ax.plot(p_range, ex_mech, label="$ex^{M}_{is}$")
ax.plot(p_range, ex_therm, label="$ex^{T}_{is}$")
ax.plot(p_range, ex, "--", label="$ex_{PH,is}$")

ax.legend()
plt.show()

We can now inspect the change of energy in the cooling process (assuming isobaric).

In [ ]:
ex_12 = ex

h_is = PropsSI("H", "P", p_range, "S", s_ref, "air")
h_3 = PropsSI("H", "P", p_range, "T", T_ref, "air")

dex_therm = []
dex_mech = []
dex = []

for p, T in zip(p_range, T_is):
    c.h.val_SI = PropsSI("H", "P", p, "T", T, "air")
    c.s.val_SI = PropsSI("S", "P", p, "T", T, "air")
    c.p.val_SI = p
    c.get_physical_exergy(p_ref, T_ref)
    ex_therm_is, ex_mech_is, ex_is = c.ex_therm, c.ex_mech, c.ex_physical
    c.h.val_SI = PropsSI("H", "P", p, "T", T_ref, "air")
    c.s.val_SI = PropsSI("S", "P", p, "T", T_ref, "air")
    c.p.val_SI = p
    c.get_physical_exergy(p_ref, T_ref)
    dex_therm += [c.ex_therm - ex_therm_is]
    dex_mech += [c.ex_mech - ex_mech_is]
    dex += [c.ex_physical - ex_is]
    

fig, ax = plt.subplots(1)

ax.set_xlabel("$\Delta p$ in $Pa$")
ax.set_ylabel("$\Delta h$ in $J/kg$")

ax.set_axisbelow(True)
ax.grid(color='gray', linestyle='dashed')

plt.tight_layout()

ax.plot(p_range, h_3 - h_is, "x-", label="$q_{2,3}$")
ax.plot(p_range, dex_therm, "o", label="$\Delta ex^{T}_{2,3}$")
ax.plot(p_range, dex_mech, label="$\Delta ex^{M}_{2,3}$")
ax.plot(p_range, dex, label="$\Delta ex^{PH}_{2,3}$")
ax.plot(p_range, ex, label="$\Delta ex^{PH}_{1,2}$")
ax.plot(p_range, [ex[i] + dex[i] for i in range(len(ex))], label="$\Delta ex^{PH}_{1,3}$")

ax.legend()
plt.show()

- Nur Kompressor vs. Kompressor mit Zwischenkühler:
  - Was ist mit dem MItteldruck
  - Nutzen der Zwischenkühlung?
  - Wem ist die Zwischenkühlung im Vergleich zuzuordnen?

- T,s/h,s-Diagramm
- Bezug Integral v dp?

## Exercises 4

Now we want to consider an improvement of our process providing the compressed air. The compressor is separated into two
individual compressor stages and the air stream is cooled in between the two compression stages.

4. Model the process of
   - compressing the ambient air to 3 bar,
   - cooling the air back to the ambient temperature and
   - compressing the air up to 10 bar.
   Assume, that the isentropic efficiency of the two compressor stages remains unchanged.
5. Balance the process with energy and exergy:
   - How does the overall power requirement change?
   - How does the overall exergy destruction change?
   - Why is the overall exergy destruction smaller?
6. Build a TESPy model for the same process.
   - Run the exergy analysis.
   - Verify the results of the models.
7. How does the intermediate pressure affect the overall exergy efficiency?
   - Calculate the exergy efficiency for an intermediate pressure range of 2 bar to 9 bar.

## Solution 3

## Solution 4

## Solution 5

## Solution 6